In [1]:
import random
import numpy as np

from torch import nn

# custom libs
from libs.PretrainedModels import AlexNet_cc, SqueezeNet_cc, ResNet18_cc, SqueezeNet1_cc
from libs.utils import get_model_name, init_model, load_model
from libs.Dataset import dst

In [2]:
random.seed(1996)
np.random.seed(1996)

## Dataset and dataLoaders

In [3]:
# parameters for dataloaders
batch_size=32
num_workers=2
drop_last=True

## First training

Start to make a training of 50 epochs to see the differences between models

In [4]:
num_epochs = 50
momentum = 0.99
criterion = nn.CrossEntropyLoss()

train_from_epoch = 0
save_each = 10
resume_global_step_from = 0

### AlexNet

In [7]:
alexnet = init_model(creator=AlexNet_cc(), model_name=get_model_name(model_name='AlexNet', lr=str(learning_rates['alexnet'])), feature_extract=True, use_pretrained=True)
alexnet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['alexnet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from)

Initializing: AlexNet__lr=0.00227
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 2.5124 Acc: 0.8704
Epoch 1/49
----------
validation Loss: 1.2292 Acc: 0.9284
Epoch 2/49
----------
validation Loss: 1.3616 Acc: 0.9310
Epoch 3/49
----------
validation Loss: 0.8292 Acc: 0.9497
Epoch 4/49
----------


### SqueezeNet 1.0

In [7]:
# to do 
squeezenet = init_model(creator=SqueezeNet_cc(), model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), feature_extract=True, use_pretrained=True)
squeezenet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

Initializing: SqueezeNet__lr=0.002
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 0.3049 Acc: 0.8925
Epoch 1/49
----------
validation Loss: 0.2364 Acc: 0.9146
Epoch 2/49
----------
validation Loss: 0.2167 Acc: 0.9230
Epoch 3/49
----------
validation Loss: 0.2004 Acc: 0.9242
Epoch 4/49
----------
validation Loss: 0.1895 Acc: 0.9333
Epoch 5/49
----------
validation Loss: 0.1715 Acc: 0.9356
Epoch 6/49
----------
validation Loss: 0.1639 Acc: 0.9432
Epoch 7/49
----------
validation Loss: 0.1671 Acc: 0.9432
Epoch 8/49
----------
validation Loss: 0.1722 Acc: 0.9333
Epoch 9/49
----------
validation Loss: 0.1421 Acc: 0.9531
Epoch 10/49
----------
validation Loss: 0.1641 Acc: 0.9455
Epoch 11/49
----------
validation Loss: 0.1306 Acc: 0.9569
Epoch 12/49
----------
validation Loss: 0

### SqueezeNet 1.1

In [ ]:
squeezenet1_1 = init_model(creator=SqueezeNet1_cc(), model_name=get_model_name(model_name='SqueezeNet1_1', lr=str(learning_rates['squeezenet1_1'])), feature_extract=True, use_pretrained=True)
squeezenet1_1.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet1_1'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

### ResNet-18

In [8]:
resnet18 = init_model(creator=ResNet18_cc(), model_name=get_model_name(model_name='ResNet18', lr=str(learning_rates['resnet18'])), feature_extract=True, use_pretrained=True)
resnet18.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['resnet18'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

Initializing: ResNet18__lr=0.00255
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 0.2604 Acc: 0.9272
Epoch 1/49
----------
validation Loss: 0.3098 Acc: 0.9333
Epoch 2/49
----------
validation Loss: 0.2485 Acc: 0.9402
Epoch 3/49
----------
validation Loss: 0.2296 Acc: 0.9501
Epoch 4/49
----------
validation Loss: 1.0046 Acc: 0.8537
Epoch 5/49
----------
validation Loss: 0.3299 Acc: 0.9375
Epoch 6/49
----------
validation Loss: 0.7834 Acc: 0.8880
Epoch 7/49
----------
validation Loss: 0.2959 Acc: 0.9417
Epoch 8/49
----------
validation Loss: 0.3085 Acc: 0.9451
Epoch 9/49
----------
validation Loss: 0.4263 Acc: 0.9211
Epoch 10/49
----------
validation Loss: 0.1576 Acc: 0.9550
Epoch 11/49
----------
validation Loss: 0.3311 Acc: 0.9314
Epoch 12/49
----------
validation Loss: 0

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Test accuracy

In [5]:
trained_models = {
    'alexnet': 'models/AlexNet__lr=0.00227/AlexNet__lr=0.00227-50.pth', 
    'squeezenet': 'models/SqueezeNet__lr=0.002/SqueezeNet__lr=0.002-50.pth',
    'squeezenet1_1': 'models/SqueezeNet1_1__lr=0.00282/SqueezeNet1_1__lr=0.00282-50.pth', 
    'resnet18': 'models/ResNet18__lr=0.00255/ResNet18__lr=0.00255-50.pth'
}

concreteCreators = {
    'alexnet': AlexNet_cc(),
    'squeezenet': SqueezeNet_cc(),
    'squeezenet1_1': SqueezeNet1_cc(),
    'resnet18': ResNet18_cc()
}

In [6]:
for name in trained_models:
    model = load_model(creator=concreteCreators[name], model_name=get_model_name(model_name=name, lr=str(learning_rates[name])), num_classes=3, model_path=trained_models[name])
    model.test_accuracy(dataset=dst)
    print("-"*10)
    print("\n\n")


Initializing: alexnet__lr=0.00227
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of alexnet__lr=0.00227: 82.93%
----------



Initializing: squeezenet__lr=0.002
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of squeezenet__lr=0.002: 86.69%
----------



Initializing: squeezenet1_1__lr=0.00282
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of squeezenet1_1__lr=0.00282: 86.05%
----------



Initializing: resnet18__lr=0.00255
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of resnet18__lr=0.00255: 80.26%
----------





## Continue training SqueezeNet 1.0 and 1.1

In [7]:
num_epochs = 50
momentum = 0.99
criterion = nn.CrossEntropyLoss()

train_from_epoch = 50
save_each = 10
resume_global_step_from = 458176 # need to continue graph on tensorboard

model = load_model(creator=concreteCreators['squeezenet1_1'], model_name=get_model_name(model_name='SqueezeNet1_1', lr=str(learning_rates['squeezenet1_1'])), num_classes=3, model_path=trained_models['squeezenet1_1'] )
model.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet1_1'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from)
model.test_accuracy(dataset=dst)

Initializing: SqueezeNet1_1__lr=0.00282
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 0.1246 Acc: 0.9630
Epoch 1/49
----------
validation Loss: 0.1209 Acc: 0.9577
Epoch 2/49
----------
validation Loss: 0.1458 Acc: 0.9531
Epoch 3/49
----------
validation Loss: 0.1291 Acc: 0.9562
Epoch 4/49
----------
validation Loss: 0.1224 Acc: 0.9558
Epoch 5/49
----------
validation Loss: 0.1269 Acc: 0.9501
Epoch 6/49
----------
validation Loss: 0.1109 Acc: 0.9623
Epoch 7/49
----------
validation Loss: 0.1218 Acc: 0.9569
Epoch 8/49
----------
validation Loss: 0.1738 Acc: 0.9398
Epoch 9/49
----------
validation Loss: 0.1304 Acc: 0.9535
Epoch 10/49
----------
validation Loss: 0.1250 Acc: 0.

## Find best model calculating accuracy

In [6]:
best_model_dir = 'models/SqueezeNet__lr=0.002'

for i in range(0,6):
    # print(i*10 + 50)
    md_name = 'SqueezeNet__lr=0.002-%s.pth' % (i*10 + 50)
    # print(md_name)
    model = load_model(creator=SqueezeNet_cc(), model_name=md_name, num_classes=3, model_path=md_name)
    model.test_accuracy(dataset=dst)
    print("-"*10)
    print("\n\n")

50
SqueezeNet__lr=0.002-50.pth
60
SqueezeNet__lr=0.002-60.pth
70
SqueezeNet__lr=0.002-70.pth
80
SqueezeNet__lr=0.002-80.pth
90
SqueezeNet__lr=0.002-90.pth
100
SqueezeNet__lr=0.002-100.pth
